In [ ]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import pandas as pd
from pandas import ExcelWriter
import spacy
from elasticsearch import helpers, Elasticsearch
import csv
import nltk
import numpy as np
from collections import Counter
import string
import re
nltk.download('stopwords')
from textblob import Word
from itertools import islice
import pandas as pd
from nltk.corpus import stopwords

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
#Define files
filename1 = 'C://Users//annac//Desktop//DAPT//Text Mining//TM Group Project//RawSurveyData2019.csv'
filename2 = 'C://Users//annac//Desktop//DAPT//Text Mining//TM Group Project//SurveyData2016-2018.csv'

In [ ]:
#read in csv's as a dataframe using pandas
newdata = pd.read_csv(filename1)
olddata = pd.read_csv(filename2)

In [ ]:
#view data
newdata.head()

In [ ]:
#view data
olddata.head()

In [ ]:
#define stop words and columns for newdata
newdata = pd.DataFrame(newdata)

#rename columns
newdata.columns = ["date_opened","category","cc_rating","resolved", "contact_customer", "cc_comments","eva_rating", "eva_comments", "cc_specialist"]

#define stopwords
stop = set(stopwords.words('english'))

#define punctuation for removal
punc = set(string.punctuation)

In [ ]:
newdata.head()

### I'm having difficulty with the below code to pull out columns 6 & 8. The current code [6:8] is targeting a range. I tried [x,x] but getting error "invalid syntax". The goal is to run the following code on the 2 text fields of survery responses (cc_comments & eva_comments), and generate sentiment and polarity scores on them.

In [ ]:
#loop through the columns in newdata, minus the first two
for col in newdata.columns[6:8]:
#create a new column for each sentiment, polarity and subjectivity, in the loop that stores sentiment values
    newdata[col+'_polarity_score'] = newdata[col].apply(lambda x: TextBlob(x).sentiment.polarity if np.all(pd.notnull(x)) else x)
    newdata[col+'_subjectivity_score'] = newdata[col].apply(lambda x: TextBlob(x).sentiment.subjectivity if np.all(pd.notnull(x)) else x)

#convert strings to lowercase and divide words into objects in a list
    newdata[col] = newdata[col].str.lower().str.split()

#check if each word is a stopword
    newdata[col] = newdata[col].apply(lambda x: [item for item in x if item not in stop] if np.all(pd.notnull(x))  else x)
    
#check for punctuation
    newdata[col] = newdata[col].apply(lambda x: [item for item in x if item not in punc] if np.all(pd.notnull(x))  else x)

#convert each phrase back into a single string (necessary for textblob to work - you could also put these in separate columns)
    newdata[col] = newdata[col].apply(lambda x: ' '.join(x) if np.all(pd.notnull(x))  else x)

### Note on above code: we can visit stemming if needed because I found this was an issue with the other dataset.

In [ ]:
newdata.head()

In [ ]:
#need to change [no answer entered] to NA's
newdata = newdata.replace({'[no answer entered]': np.nan})

In [ ]:
#there are 2 versions- [No Answer Entered] (caps) also needs to be replaced with NaN
newdata = newdata.replace({'[No Answer Entered]': np.nan})

In [ ]:
newdata.head()

### Assign new columns with interpretations of the sentiment and polarity scores. This was helpful to make charts in Kibana

In [ ]:
#on cc_comments assign positive, negative, or neutral to polarity in a new column
newdata.loc[newdata.cc_comments_polarity_score < 0, 'cc_sentiment'] = 'negative' 
newdata.loc[newdata.cc_comments_polarity_score == 0, 'cc_sentiment'] = 'neutral' 
newdata.loc[newdata.cc_comments_polarity_score > 0, 'cc_sentiment'] = 'positive'

#on eva_comments assign positive, negative, or neutral to polarity in a new column. 
newdata.loc[newdata.eva_comments_polarity_score < 0, 'eva_sentiment'] = 'negative' 
newdata.loc[newdata.eva_comments_polarity_score == 0, 'eva_sentiment'] = 'neutral' 
newdata.loc[newdata.eva_comments_polarity_score > 0, 'eva_sentiment'] = 'positive'

In [ ]:
#on cc_comments assign objective or subjective to subjectivity in a new column based on midpoint between 0-1.
newdata.loc[newdata.cc_comments_subjectivity_score <= 0.5, 'cc_subjectivity'] = 'objective' 
newdata.loc[newdata.cc_comments_subjectivity_score > 0.5, 'cc_subjectivity'] = 'subjective'

#on eva_comments assign objective or subjective to subjectivity in a new column based on midpoint between 0-1.
newdata.loc[newdata.eva_comments_subjectivity_score <= 0.5, 'eva_subjectivity'] = 'objective' 
newdata.loc[newdata.eva_comments_subjectivity_score > 0.5, 'eva_subjectivity'] = 'subjective'

### Do we need to subset each question to remove NaN's? It would make a cleaner set to analyze. I've done that to create a dataset for each question.

In [ ]:
#select columns and coll it "newdata_cc"
newdata_cc = newdata[['date_opened','category', 'resolved','cc_rating', 'cc_comments', 'cc_comments_polarity_score', 'cc_comments_sentiment_score', 'cc_sentiment', 'cc_subjectivity']]

In [ ]:
#check subset
newdata_cc.head()

In [ ]:
#drop na rows for newdata_cc_comments
newdata_cc = newdata_cc.dropna()

newdata_cc.head()

### repeat subset for second survey question eva_comments

In [ ]:
#select columns and call it newdata_eva
newdata_eva = newdata[['date_opened','category', 'resolved','eva_rating', 'eva_comments', 'eva_comments_polarity_score', 'eva_comments_sentiment_score', 'eva_sentiment', 'eva_subjectivity']]

In [ ]:
#check subset
newdata_eva.head()

In [ ]:
#drop na rows for newdata_cc_comments
newdata_eva = newdata_eva.dropna()

newdata_eva.head()

In [ ]:
#export both question sets as a csv for elasticsearch
newdata_cc.to_csv(r'C://Users//annac//Desktop//DAPT//Text Mining//newdata_cc.csv', index = False)
newdata_eva.to_csv(r'C://Users//annac//Desktop//DAPT//Text Mining//newdata_eva.csv', index = False)

In [ ]:
#Use this indexing in Kibana. This must be entered into Kibana under the "wrench" tool. 
#NOT CODE TO RUN IN PYTHON

PUT /q1
{
   "settings": {
    "analysis": {
      "analyzer": {
        "std_english": { 
          "type":      "standard",
          "stopwords": "_english_"
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "resp_id":    { "type": "keyword" },  
      "q1":  { "type": "text" ,"analyzer": "standard", "fielddata": true }, 
      "q1_polarity_score":   { "type": "integer"  },     
      "q1_subjectivity_score":   { "type": "integer"  },
      "sentiment":    { "type": "keyword" },
      "subjectivity":    { "type": "keyword" }
    }
  }
}

#Once you enter this in Kibana, THEN you can run the below code to push the data in and it will index properly.

In [ ]:
#Code used to push to elasticsearch (after index mapping above is set in Kibana)
from elasticsearch import helpers, Elasticsearch
import csv

In [ ]:
es = Elasticsearch()

In [ ]:
with open('q1.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='q1')

### We can repeat this code if we want to for the "olddata" file we imported which is from 2016-2018. NOTE: this dataset only has 1 survey question on Customer Care comments (no eva comments- that was added in 2019)

#### Once we finalize the code and all of us get the data into kibana, we can make some visualizations and screen shot them. I can show you that. 